In [1]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1']


In [2]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g", 
 "driverMemory": "8g", "proxyUser": "jchen-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [3]:
%spark add -s servicingdata -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
15164,application_1571281213559_0756,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [8]:
%%spark

print(spark.version)

2.3.0.2.6.5.0-292

In [17]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, countDistinct, max, min
from pyspark.sql import DataFrameStatFunctions as statFunc
import pyspark.sql.types as T
from functools import reduce

from os.path import expanduser, join, abspath
import time
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

spark = SparkSession.builder.getOrCreate()

In [6]:
%%spark

csv_path = '/dev/projects/retention_models/csv_data/'
#hpi_data_save_path = '/dev/projects/refi_lead_generation/datasets_old/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'
monthly_prep_path = '/dev/projects/retention_models/monthly_snapshot/monthly_preprocessed/'
actual_path = '/dev/projects/retention_models/actual_value/'
hpi_path = '/dev/projects/retention_models/hpi/'

#Refi
model_save_path_refi = '/dev/projects/retention_models/refi_payoff/models/'
result_path_refi = '/dev/projects/retention_models/refi_payoff/training/results/'

#Pur
model_save_path_pur = '/dev/projects/retention_models/purchase_payoff/models/'
result_path_pur = '/dev/projects/retention_models/purchase_payoff/training/results/movermodel/'
result_path_pur_6 = '/dev/projects/retention_models/purchase_payoff/training/results/mover_6/'
result_path_pur_12 = '/dev/projects/retention_models/purchase_payoff/training/results/mover_12/'
result_path_pur_all = '/dev/projects/retention_models/purchase_payoff/training/results/'

#prod
prod_pur_path = '/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/'
prod_refi_path = '/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/'
file_path = '/prod/projects/model_conformed/RocketScience/LeadGen/Currentfile/'

## (Before running new month) check left-portfolio data

In [7]:
%%spark

df_list = spark.read.csv(os.path.join(csv_path, 'servicing_loanlist_201909.csv'), header=True)

print(df_list.count())
df_list.show(1)

1763391
+-------------------+----------+
|servicecalendardate|     ln_no|
+-------------------+----------+
|         2019-09-30|3431477187|
+-------------------+----------+
only showing top 1 row

In [8]:
%%spark

df_book = spark.sql('''
select distinct ln_no
from data_science_sandbox.servicing_dbo_payoff_source_purchase_client_crdtrs 
where servicecalendardate = '{0}'
and leftportfoliodate is null
'''.format('2019-09-30'))

df_book.count()

1773361

In [9]:
%%spark

df1 = df_book.join(df_list, on='ln_no', how='inner')

df1.count()

1763391

## 1. Read Servicing

In [10]:
%%spark

def retrieve_dataset(servicedate, loanlist_file):
    
    spark.sql('use data_science_sandbox') 
    df_raw = spark.sql('select * from servicing_dbo_payoff_source_purchase_client_crdtrs where leftportfoliodate is null')\
                    .where(col('servicecalendardate') == servicedate)
    
    #servicing book loan list, to remove the loans already left portfolio
    df_list = spark.read.csv(os.path.join(csv_path, loanlist_file), header=True)
    
    df_ser = df_raw.join(df_list, on=['servicecalendardate', 'ln_no'], how='inner').drop_duplicates()
    
    return df_ser

In [7]:
%%spark

def retrieve_dataset_dw(servicedate):
    
    spark.sql('use data_science_sandbox') 
    df_raw = spark.sql('select * from servicing_dbo_payoff_source_purchase_client_crdtrs')\
                    .where(col('servicecalendardate') == servicedate)\
                    .drop_duplicates()
    
    return df_raw

In [11]:
%%spark

df_ser = retrieve_dataset(servicedate='2019-09-30', loanlist_file='servicing_loanlist_201909.csv')

In [8]:
%%spark

df_ser = retrieve_dataset_dw(servicedate='2016-06-30')
#df_ser.count()

In [12]:
%%spark

df_ser.groupby('servicecalendardate').agg(F.countDistinct('ln_no'), F.count('ln_no')).show()

+-------------------+---------------------+------------+
|servicecalendardate|count(DISTINCT ln_no)|count(ln_no)|
+-------------------+---------------------+------------+
|         2019-09-30|              1763391|     1763391|
+-------------------+---------------------+------------+

In [13]:
%%spark

df_ser.where((col('mosaichousehold').isNotNull()) & (col('mosaichousehold') != '')).count()

1026611

In [14]:
%%spark

df_ser.groupby('p_age_31_50').count().show()

+-----------+-------+
|p_age_31_50|  count|
+-----------+-------+
|          1| 333026|
|          6|     98|
|          3|  47974|
|          5|    937|
|          4|   8070|
|          2| 206028|
|          0|1167258|
+-----------+-------+

## 2. Preprocess Data

### (1) Selected columns

In [15]:
%%spark

df_new = df_ser.select('servicecalendardate', 'og_note_dt', 'ln_no',
                        'loantypedescription', 'unpaidbalfirstsecond', 'loanamortizationtype', 'og_mtg_am',
                        'ln_purpose_type', 'og_occupy_stat_type',
                        'ln_ann_int_rt', 'ln_tr', 'ln_monthly_pmt_am', 'ln_eloc_cd',
                        'orig_ltv_ratio_fc', 'pr_value_am', 'investornameshort',
                        'issingleborrower', 'liveyears', 'pr_zip_cd',
                        'lengthresidence', 'personnum_unit', 'estimatedhomevalue', 'mosaichousehold', 'homebedroomcnt',
                        'p1_combinedage', 'p_age_31_50', 'p_age_50more', 'p_edu_hs', 'p_married', 'quarteryearkey',
                        'mostrecentcredittriggerage', 'triggercnt1monthprior', 'triggercnt2monthprior', 'triggercnt3monthprior'
                       )

#'gcid'

### (2) Market Data

In [16]:
%%spark

def retrieve_marketdatafact():
    """Retrieve market interest rates from DL"""
    
    df = spark.sql('select * from lock2close_raw_access.marketdatafact_l2c_orc')\
                .where(F.to_date(col('dateid'), 'yyyy-MM-dd') >= '2015-12-01')\
                .withColumn('yearmonth', col('dateid').substr(1, 7))
    
    df_market = df.groupby('yearmonth')\
                    .agg(F.min('currentfannie30yryield').alias('fannie30_min'),
                         F.min('currentfannie15yryield').alias('fannie15_min'),
                         F.min('currentginny30yryield').alias('ginny30_min'))
    
    return F.broadcast(df_market)

In [17]:
%%spark

df_market_agg = retrieve_marketdatafact()

In [18]:
%%spark

df_market_agg.where(F.col('yearmonth').isin('2019-08', '2019-09', '2019-07')).show(3)

+---------+------------+------------+-----------+
|yearmonth|fannie30_min|fannie15_min|ginny30_min|
+---------+------------+------------+-----------+
|  2019-07|       2.595|       2.215|      2.348|
|  2019-08|       2.389|       2.033|      2.184|
|  2019-09|       2.298|       1.925|      2.102|
+---------+------------+------------+-----------+

### (*) Update HPI (Once necessary)

In [24]:
%%spark

hpi_raw = spark.read.csv(csv_path + 'hpi_2019q2.csv', header=True)
hpi_raw.show(1)

+---+----+-------+-----+--------------------+
|zip|year|quarter|index|                type|
+---+----+-------+-----+--------------------+
| 10|1995|      1|  100|Native 3-Digit ZI...|
+---+----+-------+-----+--------------------+
only showing top 1 row

In [25]:
%%spark

expr_mon1 = when(col('quarter')=='1', '1')\
                .otherwise(
                    when(col('quarter')=='2', '4')\
                        .otherwise(
                            when(col('quarter')=='3', '7')\
                                .otherwise('10')))
    
expr_mon2 = when(col('quarter')=='1', '2')\
                .otherwise(
                    when(col('quarter')=='2', '5')\
                        .otherwise(
                            when(col('quarter')=='3', '8')\
                                .otherwise('11')))

expr_mon3 = when(col('quarter')=='1', '3')\
                .otherwise(
                    when(col('quarter')=='2', '6')\
                        .otherwise(
                            when(col('quarter')=='3', '9')\
                                .otherwise('12')))

raw1 = hpi_raw.withColumn('month', expr_mon1)
raw2 = hpi_raw.withColumn('month', expr_mon2)
raw3 = hpi_raw.withColumn('month', expr_mon3)

hpi_new = raw1.union(raw2).union(raw3)

hpi_new.show(1)

+---+----+-------+-----+--------------------+-----+
|zip|year|quarter|index|                type|month|
+---+----+-------+-----+--------------------+-----+
| 10|1995|      1|  100|Native 3-Digit ZI...|    1|
+---+----+-------+-----+--------------------+-----+
only showing top 1 row

In [27]:
%%spark

hpi_new.write.parquet(hpi_path + 'hpi.parquet', mode='overwrite')

In [28]:
%%spark

hpi = spark.read.parquet(hpi_path + 'hpi.parquet')

hpi.show(1)

+---+----+-------+-----+--------------------+-----+
|zip|year|quarter|index|                type|month|
+---+----+-------+-----+--------------------+-----+
| 10|1995|      1|  100|Native 3-Digit ZI...|    2|
+---+----+-------+-----+--------------------+-----+
only showing top 1 row

### (3) HPI

In [19]:
%%spark

  ## Loads the HPI based on 3-digit zip (for estimating home value)## Loads 

def preprocess_hpi(pullmonth, usemax=True):
    
    # If it's for current month, set usemax=True, which means the most recent as the current value.
    # If it's for historical data, set usemax=False, which means the pull month is the current value.
       
    expr_padded_month = F.lpad(col('month'), 2, '0')
    
    df_hpi = spark.read.parquet(hpi_path + 'hpi.parquet')\
                    .withColumn('hpi_yearmonth', F.concat(col('year'), F.lit('-'), expr_padded_month))\
                    .withColumn('hpi_zip', F.lpad(col('zip'), 3, '0'))
    
    df_hpi_max_date = df_hpi.groupby('hpi_zip').agg(F.max('hpi_yearmonth').alias('yearmonth_max'))\
                            .withColumn('yearmonth_pull', F.lit(pullmonth))
    
    if not usemax:
        df_hpi_curr_date = df_hpi_max_date.withColumnRenamed('yearmonth_pull', 'hpi_yearmonth')
    else:
        df_hpi_curr_date = df_hpi_max_date.withColumnRenamed('yearmonth_max', 'hpi_yearmonth')
        
    df_hpi_current = df_hpi.join(df_hpi_curr_date, on=['hpi_yearmonth', 'hpi_zip'], how='inner')\
                        .withColumnRenamed('index', 'hpi_current')\
                        .select('hpi_zip', 'hpi_current')
    
    df_hpi = df_hpi.join(df_hpi_current, on='hpi_zip', how='left')\
                .select('hpi_zip', 'hpi_yearmonth', 'index', 'hpi_current')\
                .withColumnRenamed('index', 'hpi_og')
    
    df_hpi_avg = df_hpi.groupby('hpi_yearmonth')\
                        .agg(F.avg('hpi_og').cast('float').alias('hpi_avg'), 
                             F.avg('hpi_current').cast('float').alias('hpi_current_avg'))\
                        .withColumnRenamed('hpi_yearmonth', 'hpi_yearmonth_avg')

    return F.broadcast(df_hpi), F.broadcast(df_hpi_avg)

In [20]:
%%spark

df_hpi, df_hpi_avg = preprocess_hpi(pullmonth='2019-09', usemax=True)

#historical
#df_hpi, df_hpi_avg = preprocess_hpi(pullmonth='2016-06', usemax=False)

In [21]:
%%spark

df_hpi.where(F.col('hpi_yearmonth')=='2019-06').show(3)

+-------+-------------+------+-----------+
|hpi_zip|hpi_yearmonth|hpi_og|hpi_current|
+-------+-------------+------+-----------+
|    296|      2019-06|221.02|     221.02|
|    467|      2019-06|194.75|     194.75|
|    675|      2019-06|194.05|     194.05|
+-------+-------------+------+-----------+
only showing top 3 rows

In [58]:
%%spark

df_hpi_avg.show(3)

+-----------------+---------+---------------+
|hpi_yearmonth_avg|  hpi_avg|hpi_current_avg|
+-----------------+---------+---------------+
|          1999-10|121.28368|      230.30124|
|          2013-05|178.57294|      230.30124|
|          2009-07|183.08472|      230.30124|
+-----------------+---------+---------------+
only showing top 3 rows

### (4) Update FICO table

In [22]:
%%spark

df_fico = spark.read.csv(csv_path + 'servicing_fico_20191004.csv', header=True)\
                .select('ln_no', col('orig_fico').cast('int'))
    
df_fico.write.mode('overwrite').saveAsTable('data_science_sandbox.servicing_fico')

In [18]:
%%spark

fico_check = spark.sql('select * from data_science_sandbox.servicing_fico')

print(fico_check.count())
fico_check.show(1)

3374545
+----------+---------+
|     ln_no|orig_fico|
+----------+---------+
|3330783959|      757|
+----------+---------+
only showing top 1 row

### (5) Combine all tables

In [24]:
%%spark

def combine_tables(df, mdf = df_market_agg, df_hpi = df_hpi, df_hpi_avg=df_hpi_avg):
    
    df = df.withColumn('yearmonth', col('servicecalendardate').substr(1, 7))\
            .withColumn('og_yearmonth', col('og_note_dt').substr(1, 7))\
            .withColumn('zip3', col('pr_zip_cd').substr(1, 3))
    
    #Join market data
    df = df.join(mdf, on='yearmonth', how = 'left')
    
    #Add FICO to the table
    df_fico = spark.sql('select * from data_science_sandbox.servicing_fico')
    df = df.join(df_fico, on='ln_no', how='left')
    
    #Join the hpi data
    df = df.join(df_hpi, (df.og_yearmonth == df_hpi.hpi_yearmonth) & (df.zip3 == df_hpi.hpi_zip), how='left')\
            .withColumnRenamed('index', 'hpi_og')
            
    #Use the average to manipulate NULL hpi           
    df = df.join(df_hpi_avg, df.og_yearmonth == df_hpi_avg.hpi_yearmonth_avg, how='left')
    
    #HPI columns and HPI change
    expr_hpi_og = F.when(F.col('hpi_og').isNull(), F.col('hpi_avg')).otherwise(F.col('hpi_og'))

    expr_hpi_curr = F.when(F.col('hpi_current').isNull(), F.col('hpi_current_avg')).otherwise(F.col('hpi_current'))

    expr_hpi_change = F.when(F.col('hpi_avg').isNull(), 1.0).otherwise(expr_hpi_curr/expr_hpi_og)

    df = df.withColumn('hpi_og_exp', expr_hpi_og)\
            .withColumn('hpi_curr_exp', expr_hpi_curr)\
            .withColumn('hpi_change_exp', expr_hpi_change)
    
    return df

In [25]:
%%spark

df_all = combine_tables(df_new)

#df_all.show(1)

In [26]:
%%spark

#Save a copy for quick analysis
df_all.write.parquet(os.path.join(monthly_path, 'servicing_df_all_sep19_20191004.parquet'), mode='overwrite')

### (6). Preprocess Data Function

In [7]:
%%spark

df_all = spark.read.parquet(os.path.join(monthly_path, 'servicing_df_all_sep19_20191004.parquet'))

In [8]:
%%spark

def preprocess_dataset(df):
    
    ##credit trigger
    expr_ct_age = F.when(F.col('mostrecentcredittriggerage').isNull(), 0)\
                            .otherwise(
                                F.when(F.col('mostrecentcredittriggerage').isin(['NULL', 'null', '0']), 0)\
                                    .otherwise(F.col('mostrecentcredittriggerage') + 1
                                            )
                                    )
    expr_ct_1_mo = F.when(F.col('triggercnt1monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt1monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt1monthprior') > 1, '1')\
                                            .otherwise(F.col('triggercnt1monthprior')).cast('string')
                                            )
                                    )
                                  
    expr_ct_2_mo = F.when(F.col('triggercnt2monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt2monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt2monthprior') > 2, '2')\
                                            .otherwise(F.col('triggercnt2monthprior')).cast('string')
                                            )
                                    )
        
    expr_ct_3_mo = F.when(F.col('triggercnt3monthprior').isNull(), '0')\
                            .otherwise(
                                F.when(F.col('triggercnt3monthprior').isin(['NULL', 'null', '0']), '0')\
                                    .otherwise(
                                        F.when(F.col('triggercnt3monthprior') > 3, '3')\
                                            .otherwise(F.col('triggercnt3monthprior')).cast('string')
                                            )
                                    )
    
    ##Experian
    expr_home_value = F.when(F.col('EstimatedHomeValue').isNull(), 271225.)\
                            .otherwise(
                                F.when(F.col('EstimatedHomeValue').isin(['NULL', 'null', '0']), 271225.)\
                                    .otherwise(F.col('EstimatedHomeValue')
                                            )
                                    )
        
    expr_age_31_50 = F.when(F.col('p_age_31_50').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_age_31_50').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_age_31_50').cast('int')))
    
    expr_edu_hs = F.when(F.col('p_edu_hs').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_edu_hs').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_edu_hs').cast('int')))
    
    expr_married = F.when(F.col('p_married').isNull(), 0)\
                        .otherwise(
                            F.when(F.col('p_married').isin(['null', 'NULL']), 0)\
                                .otherwise(F.col('p_married').cast('int')))
        
    expr_bedroomcnt = F.when(F.col('homebedroomcnt').isNull(), 3)\
                            .otherwise(
                                F.when(F.col('homebedroomcnt').isin(['NULL', 'null', '0']), 3)\
                                    .otherwise(
                                        F.when(F.col('homebedroomcnt').isin(['1', '2', '3', '4', '5']), F.col('homebedroomcnt').cast('int'))\
                                            .otherwise(6)
                                            )
                                    )
                             
    expr_person_per_room = F.when(F.col('personnum_unit').isNull(), 3 / expr_bedroomcnt)\
                                .otherwise(
                                    F.when(F.col('personnum_unit').isin(['null', 'NULL', '0']), 3 / expr_bedroomcnt)\
                                        .otherwise(F.col('personnum_unit') / expr_bedroomcnt)
                                )
    
    #Different mosaic groups from the one in mover model!!
    expr_mosaicgroup = F.when(F.col('mosaichousehold').isin(['Null', 'null']), 'other')\
                            .otherwise(F.col('mosaichousehold').substr(0, 1))
        
    expr_mosaicgroup_refi_clust = F.when(expr_mosaicgroup.isin(['B', 'H', 'Q']), 'BHQ')\
                                   .otherwise(
                                       F.when(expr_mosaicgroup.isin(['A', 'C', 'G']), 'ACG')\
                                           .otherwise(
                                               F.when(expr_mosaicgroup.isin(['D', 'E', 'F', 'J', 'K']), 'DEFJK')\
                                                   .otherwise(
                                                       F.when(expr_mosaicgroup.isin(['I', 'L']), 'IL')\
                                                           .otherwise(
                                                               F.when(expr_mosaicgroup.isin(['M', 'N', 'O', 'P', 'R', 'S']), 'MNOPRS')\
                                                                   .otherwise('other')
                                                           )
                                                   )
                                               )
                                   )
        
    expr_mosaicgroup_pur_clust = F.when(expr_mosaicgroup.isin(['A', 'B']), 'AB')\
                                   .otherwise(
                                       F.when(expr_mosaicgroup.isin(['C', 'D', 'E']), 'CDE')\
                                           .otherwise(
                                               F.when(expr_mosaicgroup.isin(['F', 'G']), 'FG')\
                                                   .otherwise(
                                                       F.when(expr_mosaicgroup.isin(['I', 'J']), 'IJ')\
                                                           .otherwise('other')
                                                   )
                                            )
                                   )    
        
    ##Market Rate
    expr_rate_spread_min = F.when(F.col('ln_ann_int_rt').isNull(), 0.0)\
                                .otherwise(
                                    F.when(F.col('ln_ann_int_rt').isin(['null', 'NULL']), 0.0)\
                                        .otherwise(F.when(F.col('loantypedescription') == 'CONV', 
                                                         F.when(F.col('ln_tr') == '360', 
                                                                F.col('ln_ann_int_rt')*100 - F.col('fannie30_min'))\
                                                                .otherwise(F.col('ln_ann_int_rt')*100 - F.col('fannie15_min'))
                                                         )\
                                                   .otherwise(
                                                       F.when(F.col('loantypedescription').isin(['FHA', 'VA']), 
                                                              F.col('ln_ann_int_rt')*100 - F.col('ginny30_min'))\
                                                               .otherwise(F.col('ln_ann_int_rt')*100 - F.col('fannie30_min')))
                                                   )
                                            )

    # This is only for Mover V3 and before 
    expr_rate_spread_min_pur = F.when(F.col('ln_ann_int_rt').isNull(), 0.0)\
                                .otherwise(
                                    F.when(F.col('ln_ann_int_rt').isin(['null', 'NULL']), 0.0)\
                                        .otherwise(F.when(F.col('loantypedescription') == 'CONV', 
                                                         F.when(F.col('ln_tr') == '360', 
                                                                 F.col('fannie30_min') - F.col('ln_ann_int_rt')*100)\
                                                                .otherwise(F.col('fannie15_min') - F.col('ln_ann_int_rt')*100)
                                                         )\
                                                   .otherwise(
                                                       F.when(F.col('loantypedescription').isin(['FHA', 'VA']), 
                                                               F.col('ginny30_min') - F.col('ln_ann_int_rt')*100)\
                                                               .otherwise(F.col('fannie15_min')) - F.col('ln_ann_int_rt')*100)
                                                   )
                                            )

    expr_borrowerage_group = F.when(F.col('p1_combinedage').isNull(), 'median')\
                                .otherwise(
                                    F.when(F.col('p1_combinedage').isin(['Null', 'null']), 'median')\
                                        .otherwise(
                                            F.when(F.col('p1_combinedage').substr(2, 2).cast('double') > 40.0, 'old')\
                                                .otherwise(
                                                    F.when(F.col('p1_combinedage').substr(2, 2).cast('double') < 25.0, 'young')\
                                                        .otherwise('median')
                                                )
                                        )
                                )
    
    ##Servicing Data       
    expr_loan_type = F.when(F.col('loantypedescription').isin(['FHA', 'VA']), F.lower(F.col('loantypedescription')))\
                        .otherwise('conv')
    
    expr_ageinmon = F.when(F.col('og_note_dt').isNull(), 83)\
                        .otherwise(
                            F.when(F.col('og_note_dt').isin(['null', 'NULL']), 83)\
                                .otherwise(F.months_between(F.col('servicecalendardate'), F.col('og_note_dt')))
                        )        
    
    #Currentcltv (use HPI change)
    expr_og_mtg_am = when(col('og_mtg_am').isNull(), 173200.)\
                        .otherwise(
                            when(col('og_mtg_am').isin(['NULL', 'null']), 173200.)
                                .otherwise(
                                    col('og_mtg_am').cast('double')
                                    )
                            )

    expr_og_homevalue = when((col('pr_value_am').isNotNull()) & (~col('pr_value_am').isin(['null', 'NULL', '0', 0])), col('pr_value_am'))\
                            .otherwise(
                                when(col('orig_ltv_ratio_fc').isNull(), 0.)\
                                    .otherwise(
                                        when(col('orig_ltv_ratio_fc').isin(['null', 'NULL', '0', 0]), 0.)\
                                            .otherwise(expr_og_mtg_am / col('orig_ltv_ratio_fc'))
                                    )
                            )

    expr_currentcltv_cal = when(col('unpaidbalfirstsecond').isNull(), 0.)\
                                .otherwise(
                                    when(col('unpaidbalfirstsecond').isin(['null', 'NULL', '0']), 0.)\
                                        .otherwise(100 * col('unpaidbalfirstsecond') / (expr_og_homevalue * col('hpi_change_exp')))
                                )

    expr_currentcltv = when(expr_currentcltv_cal.isNull(), 70.8)\
                    .otherwise(
                        when(expr_currentcltv_cal > 150, 150)\
                            .otherwise(
                                when(expr_currentcltv_cal < 10, 10)\
                                    .otherwise(expr_currentcltv_cal)
                            )
                    )
    
    expr_orig_fico = F.when(F.col('orig_fico').isNull(), 735)\
                            .otherwise(
                                F.when(F.col('orig_fico').isin(['null', 'NULL']), 735)\
                                    .otherwise(
                                        F.when(F.col('orig_fico') > 850, 850)\
                                            .otherwise(
                                                F.when(F.col('orig_fico') < 500, 500)\
                                                    .otherwise(F.col('orig_fico').cast('int')
                                        )
                                    )
                                )
                            )
    
    #Liveyears
    expr_liveyears = F.when(F.col('lengthresidence').isNotNull(), F.col('lengthresidence'))\
                        .otherwise(
                            F.when(F.col('liveyears').isNotNull(), F.col('liveyears'))\
                                .otherwise(expr_ageinmon/12)
                            )
    
    expr_liveyears_short = F.when(expr_liveyears <= 4., expr_liveyears).otherwise(0.)
    
    expr_liveyears_long = F.when(expr_liveyears > 4, expr_liveyears).otherwise(0.)
                                  
    expr_liveyears_grp = F.when(expr_liveyears <= 4, 1.).otherwise(0.)                             
        
    expr_ln_purpose_type = F.when(F.col('ln_purpose_type').isin(['NULL', 'null', 'other']), 'refinance')\
                                .otherwise(F.lower(F.col('ln_purpose_type')))
    
    expr_og_occupy_stat_type = F.when(F.col('og_occupy_stat_type').isin(['NULL', 'null', 'other']), 'investment property')\
                            .otherwise(F.lower(F.col('og_occupy_stat_type')))

    expr_issingleborrower = F.when(F.col('issingleborrower').isNull(), 0)\
                                .otherwise(
                                    F.when(F.col('issingleborrower').isin(['NULL', 'null']), 0)
                                        .otherwise(F.col('issingleborrower')))
    
    expr_ln_tr = F.when(F.col('ln_tr').isNull(), 360)\
                    .otherwise(
                        F.when(F.col('ln_tr').isin(['NULL', 'null']), 360)\
                            .otherwise(F.col('ln_tr')))

        
    df = df.withColumn('ct_age_exp', expr_ct_age)\
        .withColumn('ct_1_exp', expr_ct_1_mo)\
        .withColumn('ct_2_exp', expr_ct_2_mo)\
        .withColumn('ct_3_exp', expr_ct_3_mo)\
        .withColumn('home_value_exp', expr_home_value)\
        .withColumn('p_31_50_exp', expr_age_31_50)\
        .withColumn('p_edu_hs_exp', expr_edu_hs)\
        .withColumn('p_married_exp', expr_married)\
        .withColumn('personnum_per_room_exp', expr_person_per_room)\
        .withColumn('mosaic_group_refi_exp', expr_mosaicgroup_refi_clust)\
        .withColumn('mosaic_group_pur_exp', expr_mosaicgroup_pur_clust)\
        .withColumn('ratespread_min_exp', expr_rate_spread_min)\
        .withColumn('ratespread_min_pur_exp', expr_rate_spread_min_pur)\
        .withColumn('borrowerage_bucket', expr_borrowerage_group)\
        .withColumn('loantypedescription_exp', expr_loan_type)\
        .withColumn('ageinmon_exp', expr_ageinmon)\
        .withColumn('og_mtg_am_exp', expr_og_mtg_am)\
        .withColumn('currentcltv_exp', expr_currentcltv)\
        .withColumn('orig_fico_exp', expr_orig_fico)\
        .withColumn('LiveYears_short_exp', expr_liveyears_short)\
        .withColumn('LiveYears_long_exp', expr_liveyears_long)\
        .withColumn('LiveYears_grp_exp', expr_liveyears_grp)\
        .withColumn('ln_purpose_type_exp', expr_ln_purpose_type)\
        .withColumn('og_occupy_stat_type_exp', expr_og_occupy_stat_type)\
        .withColumn('issingleborrower_exp', expr_issingleborrower)\
        .withColumn('ln_tr_exp', expr_ln_tr)\
        .withColumn('loanamortizationtype', F.lower(F.col('loanamortizationtype')))\
        .withColumn('ln_ann_int_rt', F.col('ln_ann_int_rt').cast('double'))

    return df

In [9]:
%%spark

def save_preprocess(df, filename):
    
    df_prep = preprocess_dataset(df)
    
    df1 = df_prep.select('servicecalendardate', 'ln_no', 'og_note_dt',
                           'ct_age_exp', 'ct_1_exp', 'ct_2_exp', 'ct_3_exp',
                           'home_value_exp', 'p_31_50_exp', 'p_edu_hs_exp', 'p_married_exp',
                           'personnum_per_room_exp', 'mosaic_group_refi_exp', 'mosaic_group_pur_exp',
                           'ratespread_min_exp', 'ratespread_min_pur_exp', 'ln_ann_int_rt',
                           'borrowerage_bucket', 'loantypedescription_exp', 'loanamortizationtype', 'ageinmon_exp',
                           'og_mtg_am_exp', 'currentcltv_exp', 'orig_fico_exp',
                           'LiveYears_short_exp', 'LiveYears_long_exp', 'LiveYears_grp_exp',
                           'ln_purpose_type_exp', 'ln_purpose_type', 'og_occupy_stat_type_exp', 'issingleborrower_exp',
                           'ln_tr_exp', 'investornameshort').drop_duplicates()
    
    df1.write.parquet(monthly_prep_path + filename, mode='overwrite')
    
# select('gcid')

In [10]:
%%spark

save_preprocess(df_all, 'monthly_preprocessed_201909.parquet')

In [7]:
%%spark

preprocessed_month_data = spark.read.parquet(monthly_prep_path + 'monthly_preprocessed_201909.parquet')

print(preprocessed_month_data.count())
preprocessed_month_data.show(1)

1763391
+-------------------+----------+-------------------+----------+--------+--------+--------+--------------+-----------+------------+-------------+----------------------+---------------------+--------------------+------------------+----------------------+-------------+------------------+-----------------------+--------------------+------------+-------------+-----------------+-------------+-------------------+------------------+-----------------+-------------------+---------------+-----------------------+--------------------+---------+-----------------+
|servicecalendardate|     ln_no|         og_note_dt|ct_age_exp|ct_1_exp|ct_2_exp|ct_3_exp|home_value_exp|p_31_50_exp|p_edu_hs_exp|p_married_exp|personnum_per_room_exp|mosaic_group_refi_exp|mosaic_group_pur_exp|ratespread_min_exp|ratespread_min_pur_exp|ln_ann_int_rt|borrowerage_bucket|loantypedescription_exp|loanamortizationtype|ageinmon_exp|og_mtg_am_exp|  currentcltv_exp|orig_fico_exp|LiveYears_short_exp|LiveYears_long_exp|LiveYear

## 4. Mover Refi Model Scoring

### (1) Read Models

In [13]:
%%spark

refi_model_pipeline_lr = PipelineModel.load(path = model_save_path_refi+'refiV1_3_LR_20190716')
refi_model_pipeline_rf = PipelineModel.load(path = model_save_path_refi+'refiV1_3_RF_20190716')
refi_model_pipeline_gbt = PipelineModel.load(path = model_save_path_refi+'refiV1_3_GBT_20190716')

### (2) Rename columns to score

In [14]:
%%spark

preprocessed_month_data_refi = preprocessed_month_data             
    
#.withColumnRenamed('ratespread_min_exp', 'ratespread_min_refi_exp')
#.withColumnRenamed('mosaic_group_refi_exp', 'mosaic_group_exp')

### (3) Prediction

In [15]:
%%spark

def fullMonthPrediction_refi(df):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = refi_model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', 'og_note_dt', 'investornameshort', 'loanamortizationtype', 
                                        col('loantypedescription_exp').alias('loantypedescription')
                                        col('ln_tr_exp').alias('loan_term'), 
                                        col('ageinmon_exp').alias('loan_age'), 
                                        col('currentcltv_exp').alias('currentcltv'),
                                        col('ln_ann_int_rt').alias('loan_interest_rate'), 
                                        col('pred').alias('logRegProb'))
    
    predictionsRF = refi_model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                               .select('ln_no', col('pred').alias('randForProb'))

    predictionsGBT = refi_model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))

    final_join = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                           .join(pred_df_gbt, on='ln_no', how='left')
     
    final_join = final_join.dropDuplicates()
    
    return final_join

In [16]:
%%spark

def dedup_result_pred_refi(pred_df):
    
    pred_df_dedup = pred_df.groupby(['ln_no', 'og_note_dt', 'loantypedescription', 'loanamortizationtype', 'investornameshort']).mean()
    
    oldcols = pred_df_dedup.schema.names
    newcols = ['ln_no', 'og_note_dt', 'loantypedescription', 'loanamortizationtype', 'investornameshort',\
               'loan_term', 'loan_age', 'currentcltv', 'loan_interest_rate',\
               'logRegProb', 'randForProb', 'gbtProb']
    pred_df_dedup = reduce(lambda pred_df_dedup, idx: pred_df_dedup.withColumnRenamed(oldcols[idx], newcols[idx]), range(len(oldcols)), pred_df_dedup)
    
    return pred_df_dedup

In [17]:
%%spark

def save_csv_refi(df_prep, filename, result_path = result_path_refi):
    
    pred = fullMonthPrediction_refi(df_prep)
    dedup = dedup_result_pred_refi(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

#### 2018

In [35]:
%%spark

#2018.1
pred201801_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201801_dedup_refi = dedup_result_pred_refi(pred201801_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201801_dedup_refi, 'pred_refiV1_jan18_20190409.csv')

In [52]:
%%spark

#2018.2
pred201802_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201802_dedup_refi = dedup_result_pred_refi(pred201802_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201802_dedup_refi, 'pred_refiV1_feb18_20190409.csv')

In [66]:
%%spark

#2018.4
pred201804_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201804_dedup_refi = dedup_result_pred_refi(pred201804_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201804_dedup_refi, 'pred_refiV1_apr18_20190409.csv')

In [79]:
%%spark

#2018.5
pred201805_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201805_dedup_refi = dedup_result_pred_refi(pred201805_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201805_dedup_refi, 'pred_refiV1_may18_20190409.csv')

In [91]:
%%spark

#2018.6
pred201806_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201806_dedup_refi = dedup_result_pred_refi(pred201806_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201806_dedup_refi, 'pred_refiV1_jun18_20190409.csv')

In [101]:
%%spark

#2018.7
pred201807_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201807_dedup_refi = dedup_result_pred_refi(pred201807_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201807_dedup_refi, 'pred_refiV1_jul18_20190409.csv')

In [111]:
%%spark

#2018.8
pred201808_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201808_dedup_refi = dedup_result_pred_refi(pred201808_refi)\
                            .select('ln_no', 'logRegProb', 'randForProb', 'gbtProb')

save_results_csv_refi(pred201808_dedup_refi, 'pred_refiV1_aug18_20190409.csv')

In [20]:
%%spark

#2018.12
pred201812_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201812_dedup_refi = dedup_result_pred_refi(pred201812_refi)

save_results_csv_refi(pred201812_dedup_refi, 'pred_refiV1_dec18_20190205.csv')

#### 2019

In [38]:
%%spark

#2019.1
pred201901_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201901_dedup_refi = dedup_result_pred_refi(pred201901_refi)

save_results_csv_refi(pred201901_dedup_refi, 'pred_refiV1_jan19_20190215.csv')

In [41]:
%%spark

#2019.2
pred201902_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201902_dedup_refi = dedup_result_pred_refi(pred201902_refi)

save_results_csv_refi(pred201902_dedup_refi, 'pred_refiV1_feb19_20190305.csv')

In [15]:
%%spark

#2019.3
pred201903_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201903_dedup_refi = dedup_result_pred_refi(pred201903_refi)

save_results_csv_refi(pred201903_dedup_refi, 'pred_refiV1_mar19_20190409.csv')

In [46]:
%%spark

#2019.4
pred201904_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201904_dedup_refi = dedup_result_pred_refi(pred201904_refi)

save_results_csv_refi(pred201904_dedup_refi, 'pred_refiV1_apr19_20190507.csv')

In [18]:
%%spark

#2019.5
pred201905_refi = fullMonthPrediction_refi(preprocessed_month_data_refi)
pred201905_dedup_refi = dedup_result_pred_refi(pred201905_refi)

save_results_csv_refi(pred201905_dedup_refi, 'pred_refiV1_2_may19_20190610.csv')

In [17]:
%%spark

#2019.6
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_jun19_20190717.csv')

In [46]:
%%spark

#2019.7
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_jul19_20190805.csv')

In [47]:
%%spark

#2019.8
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_aug19_20190904.csv')

In [18]:
%%spark

#2019.9
save_csv_refi(preprocessed_month_data_refi, 'pred_refiV1_3_sep19_20191004.csv')

## 5. Mover Purchase Model Scoring

### (1) Read Models

In [19]:
%%spark

pur_model_pipeline_lr = PipelineModel.load(path = model_save_path_pur + 'MoverV3_2_LR_20190718')
pur_model_pipeline_rf = PipelineModel.load(path = model_save_path_pur + 'MoverV3_2_RF_20190718')
pur_model_pipeline_gbt = PipelineModel.load(path = model_save_path_pur + 'MoverV3_2_GBT_20190718')

### (2) Rename columns to score

In [20]:
%%spark

preprocessed_month_data_pur = preprocessed_month_data

#.withColumn('loantype_exp', F.upper(F.col('loantypedescription_exp')))\
#.withColumnRenamed('ratespread_min_pur_exp', 'ratecompare_exp')\
#.withColumnRenamed('mosaic_group_pur_exp', 'mosaic_group_exp')

### (3) Prediction

In [21]:
%%spark

def fullMonthPrediction_pur(df):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = pur_model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', col('pred').alias('logRegProb'))
    
    predictionsRF = pur_model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                               .select('ln_no', col('pred').alias('randForProb'))

    predictionsGBT = pur_model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))

    final_join = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                           .join(pred_df_gbt, on='ln_no', how='left')
     
    final_join = final_join.dropDuplicates()
    
    return final_join

In [22]:
%%spark

def dedup_result_pred_pur(pred_df):
    
    pred_df_dedup = pred_df.groupby('ln_no')\
                            .agg(F.avg('logRegProb').alias('logRegProb'),
                                 F.avg('randForProb').alias('randForProb'),
                                 F.avg('gbtProb').alias('gbtProb')
                            )

    return pred_df_dedup

In [23]:
%%spark

def save_csv_pur(df_prep, filename, result_path = result_path_pur):
    
    pred = fullMonthPrediction_pur(df_prep)
    dedup = dedup_result_pred_pur(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

#### 2018

In [42]:
%%spark

#2018.1
pred201801_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201801_dedup_pur = dedup_result_pred_pur(pred201801_pur)

save_results_csv_pur(pred201801_dedup_pur, 'pred_Mover_CT3_jan18_20190409.csv')

In [54]:
%%spark

#2018.2
pred201802_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201802_dedup_pur = dedup_result_pred_pur(pred201802_pur)

save_results_csv_pur(pred201802_dedup_pur, 'pred_Mover_CT3_feb18_20190409.csv')

In [68]:
%%spark

#2018.4
pred201804_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201804_dedup_pur = dedup_result_pred_pur(pred201804_pur)

save_results_csv_pur(pred201804_dedup_pur, 'pred_Mover_CT3_apr18_20190409.csv')

In [81]:
%%spark

#2018.5
pred201805_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201805_dedup_pur = dedup_result_pred_pur(pred201805_pur)

save_results_csv_pur(pred201805_dedup_pur, 'pred_Mover_CT3_may18_20190409.csv')

In [122]:
%%spark

#2018.11
pred201811_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201811_dedup_pur = dedup_result_pred_pur(pred201811_pur)

save_results_csv_pur(pred201811_dedup_pur, 'pred_Mover_CT3_nov18_20190409.csv')

In [44]:
%%spark

#2018.12
pred201812_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201812_dedup_pur = dedup_result_pred_pur(pred201812_pur)

save_results_csv_pur(pred201812_dedup_pur, 'pred_Mover_CT3_dec18_20190131.csv')

#### 2019

In [ ]:
%%spark

#2019.1
pred201901_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201901_dedup_pur = dedup_result_pred_pur(pred201901_pur)

save_results_csv_pur(pred201901_dedup_pur, 'pred_Mover_CT3_jan19_20190215.csv')

In [47]:
%%spark

#2019.2
pred201902_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201902_dedup_pur = dedup_result_pred_pur(pred201902_pur)

save_results_csv_pur(pred201902_dedup_pur, 'pred_Mover_CT3_feb19_20190305.csv')

In [13]:
%%spark

#2019.3
pred201903_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201903_dedup_pur = dedup_result_pred_pur(pred201903_pur)

save_results_csv_pur(pred201903_dedup_pur, 'pred_Mover_CT3_mar19_20190409.csv')

In [52]:
%%spark

#2019.4
pred201904_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201904_dedup_pur = dedup_result_pred_pur(pred201904_pur)

save_results_csv_pur(pred201904_dedup_pur, 'pred_Mover_CT3_apr19_20190507.csv')

In [24]:
%%spark

#2019.5
pred201905_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201905_dedup_pur = dedup_result_pred_pur(pred201905_pur)

save_results_csv_pur(pred201905_dedup_pur, 'pred_Mover_CT3_may19_20190610.csv')

In [24]:
%%spark

#2019.6
pred201906_pur = fullMonthPrediction_pur(preprocessed_month_data_pur)
pred201906_dedup_pur = dedup_result_pred_pur(pred201906_pur)

save_results_csv_pur(pred201906_dedup_pur, 'pred_Mover_CT3_june19_20190708.csv')

In [52]:
%%spark

#2019.7
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_jul19_20190805.csv')

In [53]:
%%spark

#2019.8
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_aug19_20190904.csv')

In [24]:
%%spark

#2019.9
save_csv_pur(preprocessed_month_data_pur, 'pred_MoverV3_2_sep19_20191004.csv')

## 6. Expanded Mover Purchase Models Scoring

### (1). Read Models

In [8]:
%%spark

# 3-6 Months 
model_pipeline_lr_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_LR_20190719')
model_pipeline_rf_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_RF_20190719')
model_pipeline_gbt_6 = PipelineModel.load(path = model_save_path_pur + 'Mover_3to6_V1_GBT_20190719')

# 6-12 Months
model_pipeline_lr_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_LR_20190719')
model_pipeline_rf_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_RF_20190719')
model_pipeline_gbt_12 = PipelineModel.load(path = model_save_path_pur + 'Mover_6to12_V1_GBT_20190719')

### (2). Prediction

In [10]:
%%spark

preprocessed_month_data.printSchema()

root
 |-- servicecalendardate: date (nullable = true)
 |-- ln_no: string (nullable = true)
 |-- og_note_dt: timestamp (nullable = true)
 |-- ct_age_exp: integer (nullable = true)
 |-- ct_1_exp: string (nullable = true)
 |-- ct_2_exp: string (nullable = true)
 |-- ct_3_exp: string (nullable = true)
 |-- home_value_exp: double (nullable = true)
 |-- p_31_50_exp: integer (nullable = true)
 |-- p_edu_hs_exp: integer (nullable = true)
 |-- p_married_exp: integer (nullable = true)
 |-- personnum_per_room_exp: double (nullable = true)
 |-- mosaic_group_refi_exp: string (nullable = true)
 |-- mosaic_group_pur_exp: string (nullable = true)
 |-- ratespread_min_exp: double (nullable = true)
 |-- ratespread_min_pur_exp: double (nullable = true)
 |-- ln_ann_int_rt: double (nullable = true)
 |-- borrowerage_bucket: string (nullable = true)
 |-- loantypedescription_exp: string (nullable = true)
 |-- loanamortizationtype: string (nullable = true)
 |-- ageinmon_exp: double (nullable = true)
 |-- og_mtg

In [12]:
%%spark

def predict(model_pipeline, data):
    
    _scoreUdf = udf(lambda v: float(v[1]), DoubleType())
    
    prediction = model_pipeline.transform(data)
    pred_df = prediction.withColumn('pred', _scoreUdf(prediction['probability']))
    return pred_df

In [13]:
%%spark

def fullMonthPrediction_pur_ex(df, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt):
    
    _scoreUdf = F.udf(lambda v: float(v[1]), T.DoubleType())
    
    predictionsLR = model_pipeline_lr.transform(df)
    pred_df_lr = predictionsLR.withColumn('pred', _scoreUdf(predictionsLR['probability']))\
                                .select('ln_no', 'og_note_dt', col('pred').alias('logRegProb'))
    
    predictionsRF = model_pipeline_rf.transform(df)
    pred_df_rf = predictionsRF.withColumn('pred', _scoreUdf(predictionsRF['probability']))\
                                .select('ln_no', col('pred').alias('randForProb'))
    
    predictionsGBT = model_pipeline_gbt.transform(df)
    pred_df_gbt = predictionsGBT.withColumn('pred', _scoreUdf(predictionsGBT['probability']))\
                                .select('ln_no', col('pred').alias('gbtProb'))
    
    combinedFinalPred = pred_df_lr.join(pred_df_rf, on='ln_no', how='left')\
                                    .join(pred_df_gbt, on='ln_no', how='left')\
                                    .dropDuplicates()
    
    return combinedFinalPred

In [14]:
%%spark

def dedup_result_pred_pur_ex(pred_df):
    
    pred_df_dedup = pred_df.groupby('ln_no', 'og_note_dt')\
                            .agg(F.avg('logRegProb').alias('logRegProb'),
                                 F.avg('randForProb').alias('randForProb'),
                                 F.avg('gbtProb').alias('gbtProb')
                            )

    return pred_df_dedup

In [15]:
%%spark

def save_csv_pur(df_prep, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt, filename, result_path):
    
    pred = fullMonthPrediction_pur_ex(df_prep, model_pipeline_lr, model_pipeline_rf, model_pipeline_gbt)
    dedup = dedup_result_pred_pur_ex(pred)
    
    dedup.coalesce(1).write.csv(result_path + filename, header=True)

### (3). 3-6 Months

#### 2019

In [16]:
%%spark

# 2019-9
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_6, model_pipeline_rf_6, model_pipeline_gbt_6,
             'pred_Mover_3to6_V1_sep19_20191018.csv', 
             result_path_pur_6)

### (4). 6-12 Months

#### 2019

In [17]:
%%spark

# 2019-9
save_csv_pur(preprocessed_month_data, 
             model_pipeline_lr_12, model_pipeline_rf_12, model_pipeline_gbt_12,
             'pred_Mover_6to12_V1_sep19_20191018.csv', 
             result_path_pur_12)

## 7. Compile Mover Purchase Model Results

### (1). Compile Monthly Results

In [23]:
%%spark

def compile_scores(scorefile_3, scorefile_6, scorefile_12):
    
    df_gcid = spark.sql('select * from data_science_sandbox.gcidservicingloans')
    
    score_3 = spark.read.csv(result_path_pur + scorefile_3, header=True)\
                    .select('ln_no', 
                            col('gbtProb').cast('double').alias('score_0_3mo'))
    score_6 = spark.read.csv(result_path_pur_6 + scorefile_6, header=True)\
                    .select('ln_no', 
                            col('og_note_dt').substr(1, 10).alias('origination_date'),
                            col('gbtProb').cast('double').alias('score_3_6mo'))
    score_12 = spark.read.csv(result_path_pur_12 + scorefile_12, header=True)\
                    .select('ln_no', col('gbtProb').cast('double').alias('score_6_12mo'))
        
    score_all = score_3.join(score_6, on='ln_no', how='inner')\
                        .join(score_12, on='ln_no', how='inner')
        
    ls = []
    for score in ['score_0_3mo', 'score_3_6mo', 'score_6_12mo']:
        ls.extend(statFunc(score_all).approxQuantile(score, [0.7, 0.85, 0.95], 0.00001))
        
    # Top 5% of score_0_3mo are Mover
    # Remaining of top 15% of score_3_6mo are Searcher (~10%)
    # Remaining of top 30% of score_6_12mo are Thinker (~15%)
    expr_type = when(col('score_0_3mo') >= ls[2], 'Mover')\
                    .otherwise(
                        when(col('score_3_6mo') >= ls[4], 'Searcher')\
                            .otherwise(
                                when(col('score_6_12mo') >= ls[6], 'Thinker')\
                                    .otherwise('Other')
                            )
                    )

    score_all = score_all.withColumn('type', expr_type)\
                            .join(df_gcid, score_all.ln_no == df_gcid.Loannumber, how='left')
    
    return score_all

In [19]:
%%spark

def score_save(scorefile_3, scorefile_6, scorefile_12, servicedate, filename):
    
    score_df = compile_scores(scorefile_3, scorefile_6, scorefile_12)
    
    df1 = score_df.withColumn('servicecalendardate', lit(servicedate))\
                    .select('servicecalendardate', 'ln_no', 'GCID', 'origination_date', 
                            'type', 'score_0_3mo', 'score_3_6mo', 'score_6_12mo')
    
    df1.coalesce(1).write.csv(result_path_pur_all + filename, header=True)

In [24]:
%%spark

score_save(
    scorefile_3 = 'pred_MoverV3_2_sep19_20191004.csv',
    scorefile_6 = 'pred_Mover_3to6_V1_sep19_20191018.csv',
    scorefile_12 = 'pred_Mover_6to12_V1_sep19_20191018.csv',
    servicedate = '2019-09-30',
    filename = 'pred_Mover_Pur_All_sep19_20191018.csv'
)

### (2). Insert into the table for Dashboard

In [25]:
%%spark

t1 = spark.read.csv(result_path_pur_all + 'pred_Mover_Pur_All_sep19_20191018.csv', header=True)

print(t1.count())
t1.show(1)

1763391
+-------------------+----------+--------+----------------+-----+------------------+------------------+-------------------+
|servicecalendardate|     ln_no|    GCID|origination_date| type|       score_0_3mo|       score_3_6mo|       score_6_12mo|
+-------------------+----------+--------+----------------+-----+------------------+------------------+-------------------+
|         2019-09-30|3221449353|15625598|      2009-10-13|Mover|0.6966082344170185|0.4048104830014989|0.38521164712275213|
+-------------------+----------+--------+----------------+-----+------------------+------------------+-------------------+
only showing top 1 row

In [26]:
%%spark

t1.groupby('type').count().show()

+--------+-------+
|    type|  count|
+--------+-------+
|Searcher| 197888|
| Thinker| 288212|
|   Other|1189124|
|   Mover|  88167|
+--------+-------+

In [33]:
%%spark

# First snapshot: 9/30/2019
# Going forward, insert into the table, instead of overwriting.

t1.write.mode('overwrite').saveAsTable('Analytical.wol_dashboard_serviced_purchase_v1')

## 8. Save Results into Prod Path

### (1). Save Model Results

In [14]:
%%spark

score_refi = spark.read.csv(result_path_refi + 'pred_refiV1_3_sep19_20191004.csv', header=True)\
                    .select(col('ln_no').alias('loan'), col('gbtProb').alias('score'))
        
score_pur = spark.read.csv(result_path_pur + 'pred_MoverV3_2_sep19_20191004.csv', header=True)\
                    .select(col('ln_no').alias('loan'), col('gbtProb').alias('score'))

print(score_refi.count())
print(score_pur.count())

score_refi.show(1)
score_pur.show(1)

1763391
1763391
+----------+------------------+
|      loan|             score|
+----------+------------------+
|3349145024|0.4837531658039408|
+----------+------------------+
only showing top 1 row

+----------+-------------------+
|      loan|              score|
+----------+-------------------+
|3418610851|0.09029954693434494|
+----------+-------------------+
only showing top 1 row

In [15]:
%%spark

score_refi.coalesce(1).write.csv(prod_refi_path + 'leadgen_servicing_mover_refi_20190930.csv', header=True)
score_pur.coalesce(1).write.csv(prod_pur_path + 'leadgen_servicing_mover_pur_20190930.csv', header=True)

### (2). Save score path, and update with new version model 

In [16]:
%%spark

refi_file = spark.createDataFrame([('/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/leadgen_servicing_mover_refi_20190930.csv',)], ['filepath'])
pur_file = spark.createDataFrame([('/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/leadgen_servicing_mover_pur_20190930.csv',)], ['filepath'])

refi_file.show(10, False)
pur_file.show(10, False)

+--------------------------------------------------------------------------------------------------------------+
|filepath                                                                                                      |
+--------------------------------------------------------------------------------------------------------------+
|/prod/projects/model_conformed/RocketScience/LeadGen/RefiLeadGen/1.0/leadgen_servicing_mover_refi_20190930.csv|
+--------------------------------------------------------------------------------------------------------------+

+------------------------------------------------------------------------------------------------------------+
|filepath                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|/prod/projects/model_conformed/RocketScience/LeadGen/PurLeadGen/1.0/leadgen_servicing_mover_pur_2019

In [17]:
%%spark

# Before running, delete the two csv files in Ambari.
#file_path = '/prod/projects/model_conformed/RocketScience/LeadGen/Currentfile/'

refi_file.coalesce(1).write.csv(file_path + 'current_leadgen_servicing_mover_refi.csv', header=True)
pur_file.coalesce(1).write.csv(file_path + 'current_leadgen_servicing_mover_pur.csv', header=True)